# Modified Multiplication Table Method

## Definir fonction

import fonctions déjà définites

In [1]:
from email.utils import collapse_rfc2231_value

import sympy as sp
import numpy as np
import math
import copy

from sympy import false

global variables
variables=[]
dictionaire_auxiliary_variable={}
res=1

def variable_name(n,prefix):  # np est len de p en binaire, nq est len de q en binaire
    var_name_list = []
    for i in range(n):
        var_name_list.append(prefix+str(i+1))
    return var_name_list

def create_variables(var_name):
    if isinstance(var_name, str) :
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

def create_varibales_from_list(list):
    for element in list:
        create_variables(element)
    return

def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

def sort_key(variable):
    name = str(variable)
    # Split le nom du variable à deux parties: letter & number
    letter = name[0]
    number = int(name[1:])
    return (letter, number)

def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term

def build_rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[-1]
        rest_variable_liste.pop()
    return new_term


def IsExisted(variables_liste):
    for key, value in dictionaire_auxiliary_variable.items():
        bl = (value[0] in variables_liste) and (value[1] in variables_liste)
        if bl :
            return (key,value)
    return None

def reduct_demension_term(coef,variables_liste):
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]

    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    rest_term = build_rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    #check dict s'il existe déjà une variable auxiliary qui = les deux meme variable
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)

    if key==None:#sinon, il faut creer une variable auxiliary
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1
    else:
        x_4 = key

    reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)
    reduct_formula = sp.expand(reduct_formula)

    # mulp
    expr_reduct = rest_term * reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False

    for term in expr.args:
        # get coefficant(la constante) et variables(dans une liste)
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            if len(variables_liste) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(coef, variables_liste)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction:
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

def reduct_expression(expr_init):
    expr = sp.expand(expr_init)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_dimension_expr = reduct_demension_expression(expr_reduct_power)
    return reduct_dimension_expr

def transfers_qubo2Ising(reduct_demension_expr):
    subs_relations = {(variable, ( variable + 1 ) /2) for variable in variables}

    # Après la substitution, variables dans l'expression sont des variables d'Ising dans {-1,+1}
    new_expr = reduct_demension_expr.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

# Action

Exemple: N= 143 = 13 * 11

In [2]:
def set_variable(l,prefix):
 # l : number of bit
 # prefix : string, name of variables

    expr = 1
    var_binary = [1]
    for i in range(1,l-1):
        var_name = (prefix+ str(i))
        var = create_variables(var_name)
        expr = 2**i * var + expr
        var_binary.append(var)
    expr = expr + 2 ** (l-1)
    var_binary.append(1)
    var_binary = var_binary[::-1]
    return (expr,var_binary)

def get_N_binary(N):
    binary_str = np.binary_repr(N)
    binary_list = [int(bit) for bit in binary_str]
    return binary_list

In [3]:
(p_expr,p_binary) = set_variable(4,"p")
print(f"p_expr = {p_expr}")
print(f"p_binary = {p_binary}")

(q_expr,q_binary) = set_variable(4,"q")
print(f"q_expr = {q_expr}")
print(f"q_binary = {q_binary}")

N_binary = get_N_binary(143)
print(f"N_binary = {N_binary}")

N_binary_transposed = N_binary[::-1]
print(f"N_binary_transposed = {N_binary_transposed}")

p_expr = 2*p1 + 4*p2 + 9
p_binary = [1, p2, p1, 1]
q_expr = 2*q1 + 4*q2 + 9
q_binary = [1, q2, q1, 1]
N_binary = [1, 0, 0, 0, 1, 1, 1, 1]
N_binary_transposed = [1, 1, 1, 1, 0, 0, 0, 1]


## Table de mulp

On constuit la table de mulplication (p * q) de ligne par ligne

étape 1:

        p_binary = [1, p2, p1, 1]

        q_binary = [1, q2, q1, 1]

pour chaque ligne: un bit de q * les bits de p. On utilise 0 pour complèter
1. p * 1 = [1, p2, p1, 1]
2. p * q1 = [q1, p2*q1, p1*q1, q1, 0]

In [4]:
def table_multiplication_ligne(coef,p_binary):
    lst = [coef * element for element in p_binary]
    return lst

def table_multiplication(p_binary,q_binary):
    table = []
    lst = q_binary.copy()
    res = 0
    while not lst == []:
        element = lst.pop()
        ele = table_multiplication_ligne(element,p_binary)
        for i in range(res):
            ele.append(0)
        table.append(ele)
        res+=1
    #table.append(N_binary)
    return table

In [5]:
table = table_multiplication(p_binary,q_binary)
table

[[1, p2, p1, 1],
 [q1, p2*q1, p1*q1, q1, 0],
 [q2, p2*q2, p1*q2, q2, 0, 0],
 [1, p2, p1, 1, 0, 0, 0]]

étape 2 :
    On utilise 0 pour complèter, bien aligner

In [6]:
def adjust_table(table):
    max_cols = max(len(row) for row in table)
    complet_table = [[0]*(max_cols - len(row)) + row for row in table]
    return complet_table

In [7]:

table_complet = adjust_table(table)
table_complet

[[0, 0, 0, 1, p2, p1, 1],
 [0, 0, q1, p2*q1, p1*q1, q1, 0],
 [0, q2, p2*q2, p1*q2, q2, 0, 0],
 [1, p2, p1, 1, 0, 0, 0]]

## Obtenir chaque colonne de la table

In [8]:
def define_table_transposed(table_complet):
    table_transposed = [[row[i] for row in table_complet] for i in range(len(table_complet[0]))]
    table_transposed = table_transposed[::-1]
    return table_transposed


In [9]:
table_transposed = define_table_transposed(table_complet)
table_transposed

[[1, 0, 0, 0],
 [p1, q1, 0, 0],
 [p2, p1*q1, q2, 0],
 [1, p2*q1, p1*q2, 1],
 [0, q1, p2*q2, p1],
 [0, 0, q2, p2],
 [0, 0, 0, 1]]

## Explore la table par bloc

On divise la table par bloc. Chaque bloc contient 2 colonnes.(Pourquoi 2? Pour l'instance, on définit 2 suivant notre envie).

Par bloc, les variables(de p et q) et 2 bits de N décide le nombre de "carry" qu'on doit créer.

SommeMax : on suppose que tout les variables ont la valeur 1. Suivant les positions des variables, elle portent des poid différents( soit 2^0 ou 2^1).


On cherche une somme max qui ait les deux dernier bits que les deux bits de N, cette somme qui doit être <= SommeMax. Donc,si SommeMax n'a pas les deux meme bits que N, on peut trouvez la vraie valeur maximale en réduisant récursivement SommesMax de un.

## Définir les carry

faut par bloc

In [10]:
def num_carry(table,N_binary_transposed,first_num_col, num_col_bloc):
    expr = 0
    for i in range(num_col_bloc):
        num_col = first_num_col + i
        expr += sum(table[num_col]) * 2 ** i

    if isinstance(expr, (int)):
        max = expr
    else:
        # suppose all the terms have value 1 so that the number of the terms is the max value of the sum of the column
        variables_res = expr.free_symbols
        substitution_dict = {var: 1 for var in variables_res}
        max = expr.subs(substitution_dict)

    # ici on peux améliorer par rapport au papier: c'est de trouver un max du bloc qui a les meme dernière bits que les meme bloc de bit de N
    # binary_str = np.binary_repr(max)
    # s=binary_str[-num_col_bloc:]
    #
    # while [int(char) for char in s] != N_binary_transposed[first_num_col:first_num_col+num_col_bloc]:
    #     max -= 1
    #     binary_str = np.binary_repr(max)
    #     s=binary_str[-num_col_bloc:]

    num = len(np.binary_repr(max)) - num_col_bloc
    return num

def define_carry_var(num_carry):
    global c_res
    carry_var = []
    for i in range(num_carry):
        var = create_variables("c" + str(c_res))
        carry_var.append(var)
        c_res+=1
    return carry_var

In [11]:
num_carry(table_transposed,N_binary_transposed,1,2)

2

In [12]:
def define_carry_table(table_transposed,num_per_bloc,N_binary):
    global c_res
    c_res = 1

    #table_res = table_transposed.copy()
    table_res = copy.deepcopy(table_transposed)
    columns = len(table_transposed)

    N_binary_transposed = N_binary[::-1]
    #obj_function = 0

    #begain from the 2nd col
    first_col_num = 1

    while first_col_num < columns:
        num = num_carry(table_res,N_binary_transposed,first_col_num,num_per_bloc)
        if num > 0:
            lst_carry_var = define_carry_var(num) #a list contains the created carry variables.
            for j in range(num):
                carry_col = first_col_num + num_per_bloc + j
                if carry_col <= len(table_res)- 1:
                    table_res[carry_col].append(lst_carry_var[j])
                #elif len(table_res) < len(N_binary_transposed):
                #    table_res.append([lst_carry_var[j]])# new column

        first_col_num += num_per_bloc
    return table_res

table_carry = define_carry_table(table_transposed,2,N_binary)
table_carry

[[1, 0, 0, 0],
 [p1, q1, 0, 0],
 [p2, p1*q1, q2, 0],
 [1, p2*q1, p1*q2, 1, c1],
 [0, q1, p2*q2, p1, c2],
 [0, 0, q2, p2, c3],
 [0, 0, 0, 1, c4]]

## Define function

On peut définir la fonction objective.

Pour chque bloc, on peut définir une équation : la sommes des variables de p et q = la sommes des variables de carry et les bit de N (suivant la position de bit, les variables ont des poids différent)

In [13]:
def define_function(table_carry,num_col_per_bloc,N_binary_transposed):
    columns = len(table_carry)
    first_col_num = 1  # the 0th column is ignored
    obj_function = 0

# per bloc
    while first_col_num < columns:
        lhs = 0
        rhs = 0

        for i in range(num_col_per_bloc): # i is from 0 to num col -1
            col_num = first_col_num + i
            if col_num < len(table_carry):
                # variables of p and q
                lhs += sum(table_carry[col_num]) * 2 ** i
                if col_num < len(N_binary_transposed):
                    # bits of N
                    rhs += N_binary_transposed[col_num] * 2 ** i

                # carry
                if col_num+num_col_per_bloc < len(table_carry):
                    carry_level = table_carry[col_num + num_col_per_bloc]   # 2 is the num_col_per_bloc
                    carry_list=[]
                    for ele in carry_level:
                        if isinstance(ele,sp.Symbol):
                            if ele.name.startswith("c"):
                                carry_list.append(ele)
                    for element in carry_list:
                        rhs += element * 2**(2+i)
                else: # it's the last column of the last bloc
                    # comple the last bit of the N_binary
                    if col_num + 1 < len(N_binary_transposed):
                        rhs += N_binary_transposed[col_num + 1] * 2 ** (i + 1)

        term = (lhs - rhs)**2
        obj_function+=term

        first_col_num += num_col_per_bloc
    return obj_function

In [14]:
expr = define_function(table_carry,2,N_binary_transposed)
expr

(c3 + 2*c4 + p2 + q2 - 2)**2 + (-4*c1 - 8*c2 + 2*p1*q1 + p1 + 2*p2 + q1 + 2*q2 - 3)**2 + (c1 + 2*c2 - 4*c3 - 8*c4 + p1*q2 + 2*p1 + p2*q1 + 2*p2*q2 + 2*q1 + 1)**2

 # forme quadratique

### Créer variables auxiliaires
Avant de commencer la reduction de l'expression, on peut explorer l'expression nous avons obtenu.

Il a une structure très particulière : c'est la somme des carrés des expressions. Lorsqu'elles sont élevées au carré et développées, le résultat final doit être quadratique. On examine les expressions à l'intérieur des carrés et trouvons toutes les parties non linéaires (c'est-à-dire les termes quadratiques).

Ensuite, on peut créer des variables auxiliaires pour des parties non linéaires.

In [15]:
def find_quad_term(expr):
    square_terms = sp.sympify(expr).args
    var_pairs = []

    for square_term in square_terms:
        base = square_term.base
        terms = base.args if isinstance(base, sp.Add) else [base]
        for term in terms:
            symbols=term.free_symbols
            if len(symbols) == 2:
                var_pairs.append(symbols)

    return var_pairs

quadTerm_list = find_quad_term(expr)
quadTerm_list

[{p1, q1}, {p1, q2}, {p2, q1}, {p2, q2}]

In [16]:
# create auxiliary variables for each quadratic term
def create_aux_var_for_quadTerm(quadTerm_list):
    global res,dictionaire_auxiliary_variable

    for var_set in quadTerm_list:
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=var_set
        res += 1

res=1
dictionaire_auxiliary_variable={}
create_aux_var_for_quadTerm(quadTerm_list)

In [17]:
dictionaire_auxiliary_variable

{x1: {p1, q1}, x2: {p1, q2}, x3: {p2, q1}, x4: {p2, q2}}

### Réduction de power

In [18]:
expr = sp.expand(expr)
expr

17*c1**2 + 68*c1*c2 - 8*c1*c3 - 16*c1*c4 - 16*c1*p1*q1 + 2*c1*p1*q2 - 4*c1*p1 + 2*c1*p2*q1 + 4*c1*p2*q2 - 16*c1*p2 - 4*c1*q1 - 16*c1*q2 + 26*c1 + 68*c2**2 - 16*c2*c3 - 32*c2*c4 - 32*c2*p1*q1 + 4*c2*p1*q2 - 8*c2*p1 + 4*c2*p2*q1 + 8*c2*p2*q2 - 32*c2*p2 - 8*c2*q1 - 32*c2*q2 + 52*c2 + 17*c3**2 + 68*c3*c4 - 8*c3*p1*q2 - 16*c3*p1 - 8*c3*p2*q1 - 16*c3*p2*q2 + 2*c3*p2 - 16*c3*q1 + 2*c3*q2 - 12*c3 + 68*c4**2 - 16*c4*p1*q2 - 32*c4*p1 - 16*c4*p2*q1 - 32*c4*p2*q2 + 4*c4*p2 - 32*c4*q1 + 4*c4*q2 - 24*c4 + 4*p1**2*q1**2 + 4*p1**2*q1 + p1**2*q2**2 + 4*p1**2*q2 + 5*p1**2 + 2*p1*p2*q1*q2 + 12*p1*p2*q1 + 4*p1*p2*q2**2 + 8*p1*p2*q2 + 4*p1*p2 + 4*p1*q1**2 + 12*p1*q1*q2 - 2*p1*q1 + 6*p1*q2 - 2*p1 + p2**2*q1**2 + 4*p2**2*q1*q2 + 4*p2**2*q2**2 + 5*p2**2 + 4*p2*q1**2 + 8*p2*q1*q2 + 6*p2*q1 + 14*p2*q2 - 16*p2 + 5*q1**2 + 4*q1*q2 - 2*q1 + 5*q2**2 - 16*q2 + 14

In [19]:
expr_reduct_power = reduct_variable_power_function(expr)
expr_reduct_power

68*c1*c2 - 8*c1*c3 - 16*c1*c4 - 16*c1*p1*q1 + 2*c1*p1*q2 - 4*c1*p1 + 2*c1*p2*q1 + 4*c1*p2*q2 - 16*c1*p2 - 4*c1*q1 - 16*c1*q2 + 43*c1 - 16*c2*c3 - 32*c2*c4 - 32*c2*p1*q1 + 4*c2*p1*q2 - 8*c2*p1 + 4*c2*p2*q1 + 8*c2*p2*q2 - 32*c2*p2 - 8*c2*q1 - 32*c2*q2 + 120*c2 + 68*c3*c4 - 8*c3*p1*q2 - 16*c3*p1 - 8*c3*p2*q1 - 16*c3*p2*q2 + 2*c3*p2 - 16*c3*q1 + 2*c3*q2 + 5*c3 - 16*c4*p1*q2 - 32*c4*p1 - 16*c4*p2*q1 - 32*c4*p2*q2 + 4*c4*p2 - 32*c4*q1 + 4*c4*q2 + 44*c4 + 2*p1*p2*q1*q2 + 12*p1*p2*q1 + 12*p1*p2*q2 + 4*p1*p2 + 12*p1*q1*q2 + 10*p1*q1 + 11*p1*q2 + 3*p1 + 12*p2*q1*q2 + 11*p2*q1 + 18*p2*q2 - 11*p2 + 4*q1*q2 + 3*q1 - 11*q2 + 14

 # test

### Réduction higher-order terms
Comme nous avons déjà créé des variables auxiliaires, on peut faire la réduction avec le dictionnaire

In [20]:
# find the correspond auxiliary variable to do the reduction

def find_key(dictionary, input_set):
    for key, value in dictionary.items():
        if isinstance(value, set) and value.issubset(input_set):
            return key
    return None

x = find_key(dictionaire_auxiliary_variable, expr_reduct_power.args[40].free_symbols)
x

x4

In [21]:
abs(-1)

1

In [22]:
def select_var_for_formula(term):
    global res,dictionaire_auxiliary_variable
    x_4 = find_key(dictionaire_auxiliary_variable, term.free_symbols)
    set_symbols = term.free_symbols.copy()
    if x_4 is not None:
        set_var = dictionaire_auxiliary_variable[x_4].copy()

        new_set = set_symbols - set_var
        x_1 = set_var.pop()
        x_2 = set_var.pop()
        x_3 = new_set.pop()
    else:
        x_1 = set_symbols.pop()
        x_2 = set_symbols.pop()
        x_3 = set_symbols.pop()
        new_set = set_symbols
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]= {x_1, x_2}
        res += 1
    return x_1, x_2, x_3, x_4, new_set

def reduct_demension_term(term):
    global res

    # sélection les 3 premières variables
    x_1, x_2, x_3, x_4, new_set = select_var_for_formula(term)
    coef = term.as_coeff_mul()[0]
    coef_is_positif = coef > 0


    # garder le reste des variables
    rest_variables_liste= list(new_set)
    rest_term = build_rest_term(abs(coef), rest_variables_liste)

    if coef_is_positif:
        reduct_formula = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)
    else:
        reduct_formula = -1 * x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)
    reduct_formula = sp.expand(reduct_formula)

    # mulp
    expr_reduct = rest_term * reduct_formula
    return sp.expand(expr_reduct)

def reduct_demension_expression(expr):
    new_expr = 0
    need_reduction = False
    for term in expr.args:
        variables_set = term.free_symbols
        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_set) > 2 :
            if len(variables_set) > 3 :
                need_reduction = True
            new_term = reduct_demension_term(term)
            new_expr +=  new_term
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction:
        return reduct_demension_expression(new_expr)
    else:
        return new_expr

reduct_demension_expression(expr_reduct_power)

68*c1*c2 - 8*c1*c3 - 16*c1*c4 - 4*c1*p1 - 16*c1*p2 - 4*c1*q1 - 16*c1*q2 - 16*c1*x1 + 2*c1*x2 + 2*c1*x3 + 4*c1*x4 + 43*c1 - 16*c2*c3 - 32*c2*c4 - 8*c2*p1 - 32*c2*p2 - 8*c2*q1 - 32*c2*q2 - 32*c2*x1 + 4*c2*x2 + 4*c2*x3 + 8*c2*x4 + 120*c2 + 68*c3*c4 - 16*c3*p1 + 2*c3*p2 - 16*c3*q1 + 2*c3*q2 - 8*c3*x2 - 8*c3*x3 - 16*c3*x4 + 5*c3 - 32*c4*p1 + 4*c4*p2 - 32*c4*q1 + 4*c4*q2 - 16*c4*x2 - 16*c4*x3 - 32*c4*x4 + 44*c4 + 4*p1*p2 + 162*p1*q1 + 95*p1*q2 - 288*p1*x1 - 168*p1*x2 - 32*p1*x5 + 3*p1 + 111*p2*q1 + 142*p2*q2 + 28*p2*x1 + 12*p2*x2 - 200*p2*x3 - 248*p2*x4 - 8*p2*x5 - 11*p2 + 4*q1*q2 - 304*q1*x1 - 200*q1*x3 + 3*q1 + 12*q2*x1 - 168*q2*x2 + 12*q2*x3 - 248*q2*x4 - 11*q2 - 8*x1*x3 + 2*x1*x4 - 32*x1*x5 + 456*x1 + 252*x2 + 300*x3 + 372*x4 + 48*x5 + 14

In [23]:
dictionaire_auxiliary_variable

{x1: {p1, q1}, x2: {p1, q2}, x3: {p2, q1}, x4: {p2, q2}, x5: {p1, x1}}

In [24]:
# def reduct_demension_expression(expr):
#     term_need_reduction = multi_var_terms_list(expr)  # list of the terms which have > two variables
#
#     while term_need_reduction!=[]:
#         #create auxililary var for the most common combi
#         (x_1,x_2) = variable_combinations_counter(term_need_reduction)
#         x_4 = creat_auxiliary_var_most_common_pair((x_1,x_2))
#         new_expr = 0
#         for term in expr.args:
#             new_expr = new_expr + reduct_expression_with_dict_term(x_1,x_2,x_4,term)
#         expr = new_expr
#         term_need_reduction = multi_var_terms_list(expr)
#
#     return new_expr



In [25]:
reduct_dimension_expr = reduct_demension_expression(expr_reduct_power)
reduct_dimension_expr

68*c1*c2 - 8*c1*c3 - 16*c1*c4 - 4*c1*p1 - 16*c1*p2 - 4*c1*q1 - 16*c1*q2 - 16*c1*x1 + 2*c1*x2 + 2*c1*x3 + 4*c1*x4 + 43*c1 - 16*c2*c3 - 32*c2*c4 - 8*c2*p1 - 32*c2*p2 - 8*c2*q1 - 32*c2*q2 - 32*c2*x1 + 4*c2*x2 + 4*c2*x3 + 8*c2*x4 + 120*c2 + 68*c3*c4 - 16*c3*p1 + 2*c3*p2 - 16*c3*q1 + 2*c3*q2 - 8*c3*x2 - 8*c3*x3 - 16*c3*x4 + 5*c3 - 32*c4*p1 + 4*c4*p2 - 32*c4*q1 + 4*c4*q2 - 16*c4*x2 - 16*c4*x3 - 32*c4*x4 + 44*c4 + 4*p1*p2 + 162*p1*q1 + 95*p1*q2 - 288*p1*x1 - 168*p1*x2 - 32*p1*x5 + 3*p1 + 111*p2*q1 + 142*p2*q2 + 28*p2*x1 + 12*p2*x2 - 200*p2*x3 - 248*p2*x4 - 8*p2*x5 - 11*p2 + 4*q1*q2 - 304*q1*x1 - 200*q1*x3 + 3*q1 + 12*q2*x1 - 168*q2*x2 + 12*q2*x3 - 248*q2*x4 - 11*q2 - 8*x1*x3 + 2*x1*x4 - 32*x1*x5 + 456*x1 + 252*x2 + 300*x3 + 372*x4 + 48*x5 + 14

# Ising modèle

In [26]:
expr_ising = transfers_qubo2Ising(reduct_dimension_expr)
expr_ising

17*c1*c2 - 2*c1*c3 - 4*c1*c4 - c1*p1 - 4*c1*p2 - c1*q1 - 4*c1*q2 - 4*c1*x1 + c1*x2/2 + c1*x3/2 + c1*x4 + 41*c1/2 - 4*c2*c3 - 8*c2*c4 - 2*c2*p1 - 8*c2*p2 - 2*c2*q1 - 8*c2*q2 - 8*c2*x1 + c2*x2 + c2*x3 + 2*c2*x4 + 41*c2 + 17*c3*c4 - 4*c3*p1 + c3*p2/2 - 4*c3*q1 + c3*q2/2 - 2*c3*x2 - 2*c3*x3 - 4*c3*x4 - 3*c3/2 - 8*c4*p1 + c4*p2 - 8*c4*q1 + c4*q2 - 4*c4*x2 - 4*c4*x3 - 8*c4*x4 - 3*c4 + p1*p2 + 81*p1*q1/2 + 95*p1*q2/4 - 72*p1*x1 - 42*p1*x2 - 8*p1*x5 - 281*p1/4 + 111*p2*q1/4 + 71*p2*q2/2 + 7*p2*x1 + 3*p2*x2 - 50*p2*x3 - 62*p2*x4 - 2*p2*x5 - 223*p2/4 + q1*q2 - 76*q1*x1 - 50*q1*x3 - 281*q1/4 + 3*q2*x1 - 42*q2*x2 + 3*q2*x3 - 62*q2*x4 - 215*q2/4 - 2*x1*x3 + x1*x4/2 - 8*x1*x5 + 137*x1/2 + 81*x2/2 + 93*x3/2 + 107*x4/2 + 6*x5 + 431

# solver

In [27]:
import dimod
from dwave.preprocessing.composites import SpinReversalTransformComposite

# Compose the sampler
base_sampler = dimod.ExactSolver()
composed_sampler = SpinReversalTransformComposite(base_sampler)

def get_solver_parameter(expr_ising):
    linear = {}
    quadratic ={}
    offset = 0

    for term in expr_ising.args:
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]
        if len(term.free_symbols) ==0:
            offset = coef
        if len(term.free_symbols) ==1:
            linear[variables_liste[0].name] = coef
        elif len(term.free_symbols) ==2:
            quadratic[(variables_liste[0].name,variables_liste[1].name)] = coef

    return (linear,quadratic,offset)

def get_solution(expr_ising):
    (linear,quadratic,offset) = get_solver_parameter(expr_ising)
    response = composed_sampler.sample_ising(linear, quadratic)  #response.data() return (solution, energy, num_occurrences)
    solution = next(response.data())[0]
    return solution

In [28]:
solution = get_solution(expr_ising)
solution

{'c1': np.int8(-1),
 'x4': np.int8(-1),
 'c2': np.int8(-1),
 'x2': np.int8(1),
 'x3': np.int8(-1),
 'c4': np.int8(-1),
 'p2': np.int8(-1),
 'q2': np.int8(1),
 'p1': np.int8(1),
 'q1': np.int8(-1),
 'c3': np.int8(1),
 'x1': np.int8(-1),
 'x5': np.int8(-1)}

In [29]:
print(len(solution))

13


In [30]:
def calcule_p_q(solution,p,q):
    dic_varialbe_value = {}
    for var in variables:
        valeur_ising = solution.get(var.name, np.int8(-1))

        if(valeur_ising == np.int8(1)):
            dic_varialbe_value[var] = int(1)
        else:
            dic_varialbe_value[var] = int(0)

    p_value = p.subs(dic_varialbe_value)
    q_value = q.subs(dic_varialbe_value)

    return (p_value.evalf(),q_value.evalf())

In [31]:
calcule_p_q(solution,p_expr,q_expr)

(11.0000000000000, 13.0000000000000)

# Sum up function

In [32]:
def factorization(N,p_len,q_len,num_col_per_bloc):
    global p_expr,q_expr,N_binary,N_binary_transposed,variables, dictionaire_auxiliary_variable, res

    (p_expr,p_binary) = set_variable(p_len,"p")
    (q_expr,q_binary) = set_variable(q_len,"q")
    N_binary = get_N_binary(N)
    N_binary_transposed = N_binary[::-1]

    table = table_multiplication(p_binary,q_binary)
    table_complet = adjust_table(table)
    table_transposed = define_table_transposed(table_complet)
    table_carry = define_carry_table(table_transposed,num_col_per_bloc,N_binary)
    print(table_carry)

    expr_init = define_function(table_carry,num_col_per_bloc,N_binary_transposed)
    reduct_expr = reduct_expression(expr_init)

    expr_ising = transfers_qubo2Ising(reduct_expr)

    solution = get_solution(expr_ising)
    (p,q) = calcule_p_q(solution,p_expr,q_expr)

    return p,q

factorization(35,3,3,2)

[[1, 0, 0], [p1, q1, 0], [1, p1*q1, 1], [0, q1, p1, c1], [0, 0, 1, c2]]


(7.00000000000000, 5.00000000000000)

In [33]:
factorization(143,4,4,2)

[[1, 0, 0, 0], [p1, q1, 0, 0], [p2, p1*q1, q2, 0], [1, p2*q1, p1*q2, 1, c1], [0, q1, p2*q2, p1, c2], [0, 0, q2, p2, c3], [0, 0, 0, 1, c4]]


(13.0000000000000, 11.0000000000000)